In [1]:
import requests
import pandas as pd
from io import StringIO
from bs4 import BeautifulSoup
import re

import warnings
warnings.filterwarnings('ignore')

Tanay Station: https://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR=2023&MONTH=01&FROM=0400&TO=0400&STNM=98433

In [2]:
resp = requests.get(
    'http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR=2017&MONTH=01&FROM=0400&TO=0400&STNM=98433',
    verify = False)

resp.text

'<HTML>\n<TITLE>University of Wyoming - Radiosonde Data</TITLE>\n<BODY BGCOLOR="white">\n<H2>98433  Tanay Observations at 00Z 04 Jan 2017</H2>\n<PRE>\n-----------------------------------------------------------------------------\n   PRES   HGHT   TEMP   DWPT   RELH   MIXR   DRCT   SKNT   THTA   THTE   THTV\n    hPa     m      C      C      %    g/kg    deg   knot     K      K      K \n-----------------------------------------------------------------------------\n 1000.0    109                                                               \n  940.0    614   21.8   19.5     87  15.43     30     12  300.2  345.6  303.0\n  925.0    791   21.0   18.8     87  15.00     45     19  300.8  345.0  303.5\n  901.0   1018   19.4   17.8     90  14.46     55     35  301.4  344.2  304.0\n  865.0   1369   17.0   16.3     96  13.66     55     37  302.4  343.0  304.9\n  852.0   1499   16.4   15.6     95  13.25     59     37  303.1  342.6  305.5\n  850.0   1519   16.8   15.8     94  13.46     60     37  3

In [3]:
soup = BeautifulSoup(resp.text)
soup.findAll("pre")[0]

<pre>
-----------------------------------------------------------------------------
   PRES   HGHT   TEMP   DWPT   RELH   MIXR   DRCT   SKNT   THTA   THTE   THTV
    hPa     m      C      C      %    g/kg    deg   knot     K      K      K 
-----------------------------------------------------------------------------
 1000.0    109                                                               
  940.0    614   21.8   19.5     87  15.43     30     12  300.2  345.6  303.0
  925.0    791   21.0   18.8     87  15.00     45     19  300.8  345.0  303.5
  901.0   1018   19.4   17.8     90  14.46     55     35  301.4  344.2  304.0
  865.0   1369   17.0   16.3     96  13.66     55     37  302.4  343.0  304.9
  852.0   1499   16.4   15.6     95  13.25     59     37  303.1  342.6  305.5
  850.0   1519   16.8   15.8     94  13.46     60     37  303.7  344.0  306.2
  844.0   1580   17.0   15.2     89  13.04     61     37  304.6  343.7  306.9
  841.0   1610   18.8   13.8     73  11.93     62     36  

In [4]:
clean_soup = soup.findAll("pre")[0].contents[0].split("\n")[5:-1]
clean_soup

[' 1000.0    109                                                               ',
 '  940.0    614   21.8   19.5     87  15.43     30     12  300.2  345.6  303.0',
 '  925.0    791   21.0   18.8     87  15.00     45     19  300.8  345.0  303.5',
 '  901.0   1018   19.4   17.8     90  14.46     55     35  301.4  344.2  304.0',
 '  865.0   1369   17.0   16.3     96  13.66     55     37  302.4  343.0  304.9',
 '  852.0   1499   16.4   15.6     95  13.25     59     37  303.1  342.6  305.5',
 '  850.0   1519   16.8   15.8     94  13.46     60     37  303.7  344.0  306.2',
 '  844.0   1580   17.0   15.2     89  13.04     61     37  304.6  343.7  306.9',
 '  841.0   1610   18.8   13.8     73  11.93     62     36  306.8  342.9  309.0',
 '  829.0   1734   18.6   12.6     68  11.18     64     36  307.8  341.9  309.9',
 '  825.0   1776   19.0    7.0     46   7.67     64     35  308.7  332.4  310.1',
 '  807.0   1965   17.0   12.7     76  11.56     67     34  308.5  343.8  310.6',
 '  792.0   2124

```
 PRES   HGHT   TEMP   DWPT   RELH   MIXR   DRCT   SKNT   THTA   THTE   THTV
 hPa     m      C      C      %    g/kg    deg   knot     K      K      K 
```

In [5]:
REGEX_FORMAT = r"^" + \
               r"(?P<pres_hpa>.{8})" + \
               r"(?P<hght_m>.{6})" + \
               r"(?P<temp_c>.{7})" + \
               r"(?P<dwpt_c>.{7})" + \
               r"(?P<relh_pct>.{7})" + \
               r"(?P<mixr_gkg>.{7})" + \
               r"(?P<drct_deg>.{7})" + \
               r"(?P<sknt_knot>.{7})" + \
               r"(?P<tht_k>.{7})" + \
               r"(?P<the_k>.{7})" + \
               r"(?P<thv_k>.{7})" + \
               r".*$"
re.match(REGEX_FORMAT, clean_soup[1]).groupdict()

{'pres_hpa': '  940.0 ',
 'hght_m': '   614',
 'temp_c': '   21.8',
 'dwpt_c': '   19.5',
 'relh_pct': '     87',
 'mixr_gkg': '  15.43',
 'drct_deg': '     30',
 'sknt_knot': '     12',
 'tht_k': '  300.2',
 'the_k': '  345.6',
 'thv_k': '  303.0'}

In [6]:
for row in clean_soup[:10]:
    print(re.match(REGEX_FORMAT, row).groupdict())

{'pres_hpa': ' 1000.0 ', 'hght_m': '   109', 'temp_c': '       ', 'dwpt_c': '       ', 'relh_pct': '       ', 'mixr_gkg': '       ', 'drct_deg': '       ', 'sknt_knot': '       ', 'tht_k': '       ', 'the_k': '       ', 'thv_k': '       '}
{'pres_hpa': '  940.0 ', 'hght_m': '   614', 'temp_c': '   21.8', 'dwpt_c': '   19.5', 'relh_pct': '     87', 'mixr_gkg': '  15.43', 'drct_deg': '     30', 'sknt_knot': '     12', 'tht_k': '  300.2', 'the_k': '  345.6', 'thv_k': '  303.0'}
{'pres_hpa': '  925.0 ', 'hght_m': '   791', 'temp_c': '   21.0', 'dwpt_c': '   18.8', 'relh_pct': '     87', 'mixr_gkg': '  15.00', 'drct_deg': '     45', 'sknt_knot': '     19', 'tht_k': '  300.8', 'the_k': '  345.0', 'thv_k': '  303.5'}
{'pres_hpa': '  901.0 ', 'hght_m': '  1018', 'temp_c': '   19.4', 'dwpt_c': '   17.8', 'relh_pct': '     90', 'mixr_gkg': '  14.46', 'drct_deg': '     55', 'sknt_knot': '     35', 'tht_k': '  301.4', 'the_k': '  344.2', 'thv_k': '  304.0'}
{'pres_hpa': '  865.0 ', 'hght_m': '  13

In [7]:
import datetime as dt
import time

URL_FORMAT = "http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR={year}&MONTH={month}&FROM={day}00&TO={day}00&STNM=98433"

start_date = dt.datetime(2017, 1, 1)
end_date = dt.datetime.now()
delta = dt.timedelta(days=1)

while start_date <= end_date:
    station_data = []

    url = URL_FORMAT.format(
        year=start_date.strftime("%Y"),
        month=start_date.strftime("%m"),
        day=start_date.strftime("%d"))

    resp = requests.get(url, verify=False)
    
    
    try:
        soup = BeautifulSoup(resp.text)
        soup.findAll("pre")[0]
        clean_soup = soup.findAll("pre")[0].contents[0].split("\n")[5:-1]

        for row in clean_soup:
            row_obj = re.match(REGEX_FORMAT, row).groupdict()
            station_data.append(row_obj)

        station_data_df = pd.DataFrame.from_dict(station_data)

        # Remove spaces
        for col in station_data_df.columns:
            station_data_df[col] = station_data_df[col].str.strip()

        station_data_df.to_csv("data/output/Upper Air Data - UWYO/UWYO-UPPER-AIR-{year}-{month}-{day}.csv".format(
            year=start_date.strftime("%Y"),
            month=start_date.strftime("%m"),
            day=start_date.strftime("%d")))
    except:
        print("Error: " + url)

    start_date += delta
    


Error: http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR=2017&MONTH=01&FROM=0200&TO=0200&STNM=98433
Error: http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR=2017&MONTH=03&FROM=0500&TO=0500&STNM=98433
Error: http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR=2017&MONTH=04&FROM=1800&TO=1800&STNM=98433
Error: http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR=2017&MONTH=06&FROM=0800&TO=0800&STNM=98433
Error: http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR=2017&MONTH=07&FROM=3000&TO=3000&STNM=98433
Error: http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR=2017&MONTH=07&FROM=3100&TO=3100&STNM=98433
Error: http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR=2017&MONTH=08&FROM=0100&TO=0100&STNM=98433
Error: http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR=2017&MONTH=08&FROM=0200&TO=

Error: http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR=2020&MONTH=10&FROM=2600&TO=2600&STNM=98433
Error: http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR=2020&MONTH=11&FROM=1400&TO=1400&STNM=98433
Error: http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR=2020&MONTH=12&FROM=0600&TO=0600&STNM=98433
Error: http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR=2020&MONTH=12&FROM=0700&TO=0700&STNM=98433
Error: http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR=2021&MONTH=02&FROM=0200&TO=0200&STNM=98433
Error: http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR=2021&MONTH=05&FROM=0300&TO=0300&STNM=98433
Error: http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR=2021&MONTH=05&FROM=0600&TO=0600&STNM=98433
Error: http://weather.uwyo.edu/cgi-bin/sounding?region=seasia&TYPE=TEXT%3ALIST&YEAR=2021&MONTH=06&FROM=0300&TO=

In [8]:
station_data_df

,pres_hpa,hght_m,temp_c,dwpt_c,relh_pct,mixr_gkg,drct_deg,sknt_knot,tht_k,the_k,thv_k
0,942.0,614,20.0,19.7,98,15.59,70,10,298.2,343.6,301.0
1,915.0,874,19.0,19.0,100,15.36,70,21,299.7,344.7,302.4
2,882.0,1202,18.2,18.2,100,15.15,70,34,302.0,346.9,304.7
3,871.0,1314,17.0,16.8,99,13.98,70,39,301.8,343.2,304.3
4,866.0,1365,16.4,16.1,98,13.47,69,38,301.7,341.6,304.1
5,858.0,1447,17.2,17.2,100,14.61,67,38,303.3,346.9,306.0
6,850.0,1530,15.8,15.8,100,13.46,65,37,302.7,342.7,305.1
7,795.0,2099,14.4,14.4,100,13.14,100,30,307.0,346.9,309.4
8,744.0,2658,12.6,11.1,91,11.31,135,23,311.0,345.9,313.1
9,727.0,2854,12.0,10.0,88,10.72,135,25,312.4,345.7,314.4
